# Light under sea ice

The notebook demonstrates estimating photosynthetically active radiation (PAR) under sea ice following @stroeve_multi-sensor_2021.  The approach of @stroeve_multi-sensor_2021 is based on that of @maykut1971 and @grenfell_optical_1977.  The approach uses apparent optical properties (albedo $\alpha$ and attenuation coefficient $\kappa$) in a two level Bouger-Lambert-Beer exponential decay model.  The first level represents the surface scattering layer - not included for dry snow.  The second level is snow-ice column.

_Need a description of what the snow sea ice column looks like with a diagram and maybe some images_ 

The model is implemented here, first, using basic numpy functions.  It is then extended to account for distributions of ice thickness and snow depth.  

In [1]:
import numpy as np

from beer_lambert_rt.constants import (hssl_ice, hssl_dry_snow, hssl_wet_snow,
                                       k_ice, k_dry_snow, k_wet_snow, k_thin_wet_snow,
                                       i0_ice, i0_dry_snow, i0_wet_snow, i0_melt_ponds,
                                       albedo_open_water)
import beer_lambert_rt.transmission as transmission
from beer_lambert_rt.original_functions import get_f_att_snow

## Bouger-Lambert-Beer Model

The Bouger-Lambert-Beer model is an approximation of the Radiative Transfer Equation that assumes all attenuation in a medium occurs by absorption.  Single scattering and multiple scattering terms vanish.  Radiative flux $F$ at the base of a column of depth $z$ is given by;

$$
F = F_{0}\exp{(- \kappa z)}
$$ {#eq-expontential_attenuation}

where $F_{0}$ is the radiative flux at the upper surface of the medium of interest and $\kappa$ is the attenuation coefficient of the medium.

Sea ice may be covered by snow.  Furthermore, wet snow and bare sea ice have a surface scattering layer (SSL) that may be up to 10 cm in depth.  Incoming solar radiation is absorbed (or scattered) by this SSL.  @maykut1971 account for this reduction in radiation transmitted through the SSL with a surface transmission function $i_{0}$.  Radiation is then attenuated by the snow (if present) and ice layers below the SSL.  Snow and ice have different attenuation coefficients (_Need Ref_).  The transmission coefficient $T_{ocean}$ for a snow covered ice is the product of two exponential terms, one for snow and one for ice.

$$
T_{ocean} = i_{0,snow} (1 - \alpha_{snow}) \exp(-\kappa_{snow}(h_{snow} - h_{ssl})) \exp(-\kappa_{ice} h_{ice})
$$

where $i_{0,snow}$ is the surface transmission parameter for snow, $\alpha_{snow}$ is the snow albedo, $\kappa_{snow}$ and $\kappa_{ice}$ are attenuation coeffiecnts for snow and ice, and $h_{snow}$, $h_{ssl}$ and $h_{ice}$ are snow depth, thickness of the SSL, and ice thickness respectively.

For bare ice

$$
T_{ocean} = i_{0,ice} (1 - \alpha_{ice}) \exp(-\kappa_{ice} (h_{ice} - h_{ssl}))
$$

Radiative flux at the ice-ocean interface is then

$$
F_{ocean} = F_{0} T_{ocean}
$$


### Implementation

The code cell below demonstrates how the Beer-Lambert RT modelis implemented.  

Rather than having separate functions for each situation or having a complex nest of _if-elif-else_ conditional blocks, parameters are first selected based on snow depth (if present), skin temperature, melt pond depth (if present) and ice thickness.  These are then passed to a pair of common of exponential decay functions: one for snow and one for ice.

$$
esnow = \exp{[-\kappa_{snow}(h_{snow} - h_{ssl})]}
$$

and

$$
eice = \exp{[-\kappa_{ice}(h_{ice} - h_{ssl})]}
$$

$esnow$ evaluates to 1 if $h_{snow}$ is zero.  $eice$ could also evaulate to 1. if $h_{ice}$ is also zero.  But this situation will raise a `ValueError` in the code.

The surface transmission parameter $i_0$ is also selected based on surface conditions (surface type: base ice, melting snow, dry snow or pond) and temperature.  The total transmittance for the snow-ice-pond column is the product of $esnow$, $eice$ and $i_0$.

This approach has several benefits.  First, it is clearer because it avoids the _if-elif-else_ structures.  Second it is more efficient.  Conditionals take time to evaluate.  I utilize the `numpy.piecewise` and `numpy.select` functions.  This allows arrays for `hce`, `hsnow`, `hpond` and `surface_temperature` to be passed and evaulated at once.  Third, it is easier to maintain and make changes to the code.  Functions to select parameters can be modified individually without having to modify the transmittance function.  There is also the possibility to have methods to select parameterizations, as is demonstrated by the surface scattering parameterizations: `green_edge_hssl_*` and `cice_hssl_*`.

In [2]:
hice = 1.5
hsnow = 0.
hpond = 0.
surface_temperature = 0.

i0 = transmission.select_surface_transmission(hice, hsnow, hpond, surface_temperature)
hssl_ice = transmission.green_edge_hssl_ice(hice, hsnow, hpond)
hssl_snow = transmission.green_edge_hssl_snow(hsnow, surface_temperature)
kice = transmission.select_attenuation_ice(hice)
ksnow = transmission.select_attenuation_snow(hsnow, surface_temperature)

# Evaluates to zero when hsnow is zero
esnow = np.exp(-1. * ksnow * (hsnow - hssl_snow))

eice = np.exp(-1. * kice * (hice - hssl_ice))

transmittance = i0 * esnow * eice

print(f"hsnow: {hsnow}")
print(f"hice: {hice}")
print(f"hpond: {hpond}")
print(f"surface_temperature: {surface_temperature}")
print("")
print(f"Surface type: {transmission.surface_type(hice, hsnow, hpond, surface_temperature)}")
print(f"Surface transmission (i0): {i0}")
print(f"SSL thickness ice: {hssl_ice}")
print(f"SSL thickness snow: {hssl_snow}")
print(f"Attenuation ice (kice): {kice}")
print("")
print("Calculate transmittance")
print(f"Snow component of transmittance: {esnow:5.4f}")
print(f"Ice component of transmittance: {eice:5.4f}")
print(f"Total transmittance: {transmittance:5.4f}")

hsnow: 0.0
hice: 1.5
hpond: 0.0
surface_temperature: 0.0

Surface type: Bare ice
Surface transmission (i0): 0.26
SSL thickness ice: 0.1
SSL thickness snow: 0.0
Attenuation ice (kice): 1.0

Calculate transmittance
Snow component of transmittance: 1.0000
Ice component of transmittance: 0.2466
Total transmittance: 0.0641


## Running the tranmittance function

This section compares the refactored transmittance function with the original transmittance function `get_f_att_snow`.  In spite of the name, the original function calculates transmittance for snow covered ice, when the snow is dry and wet, and bare ice.  It does not calculate transmittance for melt ponds.

The new refactored code calculates transmittance for snow covered and bare ice conditions, and for melt ponds.

N.B. The equations for transmittance (see above) include albedo.  Neither the original code nor the refactored code include albedo in the calculations.

The original and refactored code are tested on 8 cases that capture the parameterizations of surface transmission, snow and ice attenuation coefficients, and surface scattering layer.  These test cases are in the `SURFACE_CONDITION` variable.  These a printed below

In [6]:
from tests.test_transmission import SURFACE_CONDITION

def unpack_case(case):
    """Gets the variables for a case and returns a tuple of numpy.arrays"""
    return (np.array(case[varname]) for varname in ["hice", "hsnow", "hpond", "skin_temperature"])

In [12]:
for key, case in SURFACE_CONDITION.items():
    hice, hsnow, hpond, skin_temperature = unpack_case(case)
    print(f"{' '.join(key.split('_')).title()+':':17s} hsnow={hsnow:4.2f}, hice={hice:4.2f}, hpond={hpond:3.1f}, skin_temp={skin_temperature:5.1f}") 

Dry Snow:         hsnow=0.30, hice=1.50, hpond=0.0, skin_temp=-10.0
Wet Snow:         hsnow=0.30, hice=1.50, hpond=0.0, skin_temp=  0.0
Thin Wet Snow:    hsnow=0.01, hice=1.50, hpond=0.0, skin_temp=  0.0
Thick Bare Ice:   hsnow=0.00, hice=1.50, hpond=0.0, skin_temp=  0.0
Medium1 Bare Ice: hsnow=0.00, hice=0.70, hpond=0.0, skin_temp=  0.0
Medium2 Bare Ice: hsnow=0.00, hice=0.30, hpond=0.0, skin_temp=  0.0
Thin Bare Ice:    hsnow=0.00, hice=0.09, hpond=0.0, skin_temp=  0.0
Melt Pond:        hsnow=0.00, hice=1.50, hpond=0.3, skin_temp=  0.0


In [14]:
for key, case in SURFACE_CONDITION.items():
    hice, hsnow, hpond, skin_temperature = unpack_case(case)
    trans_new = transmission.calculate_transmittance(hice, hsnow, hpond, skin_temperature)
    trans_orig = get_f_att_snow(hsnow, hice, skin_temperature)
    print(f"{key:16s}: {trans_new:5.4f} {trans_orig:5.4f}")

dry_snow        : 0.0273 0.0273
wet_snow        : 0.0260 0.0273
thin_wet_snow   : 0.0673 0.2080
thick_bare_ice  : 0.0641 0.0641
medium1_bare_ice: 0.1380 0.1380
medium2_bare_ice: 0.7408 0.7408
thin_bare_ice   : 0.3396 0.9139
melt_pond       : 0.1250 0.0641


## Reasons for differences

There are differences for the wet_snow, thin_wet_snow, and thin_bare_ice cases.  The reasons for these differences are given below.

wet_snow - original code tests for temperature greater than zero  
For hice < 0.5 there is no surface scattering layer and i0_ice is 1.  - Fixed in my code  
thin_bare_ice - The logic in the original code does not allow the hice < 0.1 to behave as expected.  Any ice with hice < 0.5 goes to the first case.  - May need to demonstrate this.  
melt_ponds - get_f_att_snow does not deal with melt ponds  
